In [2]:
import pandas as pd
import numpy as np
import scipy.optimize as optimize
import os
import matplotlib.pyplot as plt
import torch

In [3]:
%matplotlib widget
plt.rcParams["figure.autolayout"] = True
plt.rcParams["figure.figsize"] = [7,5]

# Merging Data

In [4]:
employment = pd.read_csv('../BLS_EmployDelta.csv')
employment.drop(['Unnamed: 0'], axis = 1, inplace = True)
employment.head()

,Month,Total nonfarm,Total private,Mining and logging,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Utilities,Information,Financial activities,Professional and business services,Private education and health services,Leisure and hospitality,Other services,Government,Federal government,State government,Local government
0,2003-10-01,0.0,0.001223,0.005263,0.000147,-0.000906,-0.000054,0.001254,0.002210,-0.000174,-0.002214,-0.001235,0.001366,0.003495,0.004178,-0.000185,0.002511,-0.002546,0.001406,0.003920
1,2003-11-01,0.0,0.000606,0.005236,0.001769,-0.001256,0.000379,-0.001607,0.000839,-0.002618,-0.001585,-0.000865,0.002356,0.002125,0.002039,0.000370,-0.001067,-0.005106,0.000201,-0.000723
2,2003-12-01,0.0,0.000936,-0.001736,0.004562,-0.001118,0.002202,-0.000168,-0.003162,-0.001225,0.000000,-0.000866,0.001670,0.002238,0.002931,-0.000185,0.000511,0.004399,-0.001403,0.000434
3,2004-01-01,0.0,0.001632,0.001739,0.003076,-0.000699,0.002054,0.002892,0.007233,-0.001051,-0.002857,0.000248,0.001791,0.001881,0.002110,0.000370,-0.000371,-0.002920,-0.003212,0.001157
4,2004-02-01,0.0,0.000412,0.001736,-0.001460,-0.000770,0.001025,-0.000060,0.001097,-0.002631,0.000000,0.000000,0.000801,0.001056,0.002106,0.000000,0.000557,-0.000366,0.000403,0.000795


In [5]:
banks = pd.read_csv('../commercial_banks_data.csv')
banks.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [6]:
banks[-48:-36]

,DATE,borrowings_pc,consumer_loans_pc,commercial_loans_pc
189,2019-06-01,-0.17964,5.2,3.0
190,2019-07-01,1.74497,4.6,4.0
191,2019-08-01,-0.03699,7.6,9.7
192,2019-09-01,-0.18107,4.2,3.7
193,2019-10-01,-1.22205,3.7,-0.2
194,2019-11-01,-0.36796,3.3,-1.5
195,2019-12-01,1.09588,4.4,-6.7
196,2020-01-01,-1.61515,6.6,-2.4
197,2020-02-01,-0.20918,4.8,-2.2
198,2020-07-01,-2.58517,1.3,-28.2


In [7]:
deficit= pd.read_csv('../filtered_deficit.csv')
deficit.drop(['Unnamed: 0'], axis = 1, inplace = True)
pc = (np.array(deficit['Deficit_mil'][1:]) - np.array(deficit['Deficit_mil'][:-1]))/np.array(deficit['Deficit_mil'][:-1])
deficit['Percent_change'] = np.concatenate([np.array([np.nan]), pc])

In [8]:
deficit[-48:-36]

,Month,Deficit_mil,Percent_change
464,2019-06-01,-8476.318454,-0.959203
465,2019-07-01,-119695.806193,13.121202
466,2019-08-01,-200344.348929,0.673779
467,2019-09-01,82767.658465,-1.413127
468,2019-10-01,-134467.564934,-2.624639
469,2019-11-01,-208838.294457,0.553076
470,2019-12-01,-13272.557316,-0.936446
471,2020-01-01,-32593.280008,1.455690
472,2020-02-01,-235335.213875,6.220360
473,2020-07-01,-62991.726395,-0.732332


In [9]:
itb = pd.read_csv('../Index_Fund_ITB_Cleaned.csv')
itb.drop(['Unnamed: 0'], axis = 1, inplace = True)
itb.drop(np.arange(166, 170), axis=0, inplace = True)
itb.reset_index(drop = True, inplace = True)
itb

,Date,Open,High,Low,Close,Adj Close,Volume
0,2006-05-01,45.986471,46.060000,45.517647,45.557059,40.369449,1.367059e+04
1,2006-06-01,39.310455,39.742273,38.888182,39.294091,34.829539,1.447273e+04
2,2006-07-01,35.906500,36.288500,35.403500,35.793500,31.746150,4.817500e+04
3,2006-08-01,35.043913,35.553913,34.658696,35.147391,31.173101,8.652609e+04
4,2006-09-01,36.653000,37.353500,36.140500,36.796000,32.647215,2.925850e+05
...,...,...,...,...,...,...,...
202,2023-07-01,86.801500,87.440000,86.017501,86.774500,86.608764,2.440115e+06
203,2023-08-01,86.121739,86.913479,85.137825,86.016957,85.852666,3.464996e+06
204,2023-09-01,82.167000,82.827500,81.192999,81.800000,81.673698,3.719340e+06
205,2023-10-01,74.901363,75.763183,74.125000,74.803182,74.803182,3.397895e+06


In [10]:
vgt = pd.read_csv('../Index_Fund_VGT_Cleaned.csv')
vgt.drop(['Unnamed: 0'], axis = 1, inplace = True)
vgt.drop(np.arange(193, 197), axis=0, inplace = True)
vgt.reset_index(drop = True, inplace = True)
vgt

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-02-01,48.210000,48.304210,47.976842,48.111579,40.772834,28515.789474
1,2004-03-01,45.919130,46.046521,45.733913,45.861739,38.866178,9539.130435
2,2004-04-01,46.551428,46.617143,46.307619,46.413334,39.333634,3419.047619
3,2004-05-01,44.138000,44.241000,44.059000,44.130000,37.398592,1540.000000
4,2004-06-01,45.818095,45.892381,45.728095,45.812857,38.824751,785.714286
...,...,...,...,...,...,...,...
229,2023-07-01,447.110002,450.020503,444.202003,447.172003,446.212662,603715.000000
230,2023-08-01,431.790870,434.993043,427.986957,431.346523,430.421135,589060.869565
231,2023-09-01,427.936003,430.229997,424.484000,426.990500,426.163295,476455.000000
232,2023-10-01,416.755001,420.504544,413.025454,416.710001,416.710001,540877.272727


In [11]:
sp500 = pd.read_csv('../S&P500_Cleaned.csv')
sp500.drop(['Unnamed: 0'], axis = 1, inplace = True)
sp500.drop(np.arange(206, 210), axis=0, inplace = True)
sp500.reset_index(drop = True, inplace = True)
sp500

,Month,Open,High,Low,Close,Adj Close,Volume
0,2003-01-01,879.820007,935.049988,840.340027,855.700012,855.700012,30749580000
1,2003-02-01,855.700012,864.640015,806.289978,841.150024,841.150024,25235300000
2,2003-03-01,841.150024,895.900024,788.900024,848.179993,848.179993,30080030000
3,2003-04-01,848.179993,924.239990,847.849976,916.919983,916.919983,29669610000
4,2003-05-01,916.919983,965.380005,902.830017,963.590027,963.590027,30952100000
...,...,...,...,...,...,...,...
242,2023-07-01,4450.479980,4607.069824,4385.049805,4588.959961,4588.959961,75063200000
243,2023-08-01,4578.830078,4584.620117,4335.310059,4507.660156,4507.660156,86840820000
244,2023-09-01,4530.600098,4541.250000,4238.629883,4288.049805,4288.049805,73482980000
245,2023-10-01,4284.520020,4393.569824,4103.779785,4117.370117,4117.370117,71792290000


In [12]:
for df in [itb, vgt, sp500]:
    df['Avg_price'] = (np.array(df['Open']) + np.array(df['Close']))/2
    dP = (np.array(df['Avg_price'][1:]) - np.array(df['Avg_price'][:-1]))/np.array(df['Avg_price'][:-1])
    df['PC_change_price'] = np.concatenate([np.array([np.nan]), dP])
    dV = (np.array(df['Volume'][1:]) - np.array(df['Volume'][:-1]))/np.array(df['Volume'][:-1])
    df['PC_change_vol'] = np.concatenate([np.array([np.nan]), dV])
itb.to_csv('../Index_Fund_ITB_Cleaned_1.csv')
vgt.to_csv('../Index_Fund_VGT_Cleaned_1.csv')
sp500.to_csv('../S&P500_Cleaned_1.csv')

In [13]:
vgt

,Date,Open,High,Low,Close,Adj Close,Volume,Avg_price,PC_change_price,PC_change_vol
0,2004-02-01,48.210000,48.304210,47.976842,48.111579,40.772834,28515.789474,48.160789,NaN,NaN
1,2004-03-01,45.919130,46.046521,45.733913,45.861739,38.866178,9539.130435,45.890435,-0.047141,-0.665479
2,2004-04-01,46.551428,46.617143,46.307619,46.413334,39.333634,3419.047619,46.482381,0.012899,-0.641577
3,2004-05-01,44.138000,44.241000,44.059000,44.130000,37.398592,1540.000000,44.134000,-0.050522,-0.549582
4,2004-06-01,45.818095,45.892381,45.728095,45.812857,38.824751,785.714286,45.815476,0.038099,-0.489796
...,...,...,...,...,...,...,...,...,...,...
229,2023-07-01,447.110002,450.020503,444.202003,447.172003,446.212662,603715.000000,447.141003,0.040382,-0.028132
230,2023-08-01,431.790870,434.993043,427.986957,431.346523,430.421135,589060.869565,431.568696,-0.034826,-0.024273
231,2023-09-01,427.936003,430.229997,424.484000,426.990500,426.163295,476455.000000,427.463251,-0.009513,-0.191162
232,2023-10-01,416.755001,420.504544,413.025454,416.710001,416.710001,540877.272727,416.732501,-0.025103,0.135212


In [14]:
itb

,Date,Open,High,Low,Close,Adj Close,Volume,Avg_price,PC_change_price,PC_change_vol
0,2006-05-01,45.986471,46.060000,45.517647,45.557059,40.369449,1.367059e+04,45.771765,NaN,NaN
1,2006-06-01,39.310455,39.742273,38.888182,39.294091,34.829539,1.447273e+04,39.302273,-0.141342,0.058676
2,2006-07-01,35.906500,36.288500,35.403500,35.793500,31.746150,4.817500e+04,35.850000,-0.087839,2.328675
3,2006-08-01,35.043913,35.553913,34.658696,35.147391,31.173101,8.652609e+04,35.095652,-0.021042,0.796079
4,2006-09-01,36.653000,37.353500,36.140500,36.796000,32.647215,2.925850e+05,36.724500,0.046412,2.381466
...,...,...,...,...,...,...,...,...,...,...
202,2023-07-01,86.801500,87.440000,86.017501,86.774500,86.608764,2.440115e+06,86.788000,0.079389,0.066784
203,2023-08-01,86.121739,86.913479,85.137825,86.016957,85.852666,3.464996e+06,86.069348,-0.008281,0.420013
204,2023-09-01,82.167000,82.827500,81.192999,81.800000,81.673698,3.719340e+06,81.983500,-0.047472,0.073404
205,2023-10-01,74.901363,75.763183,74.125000,74.803182,74.803182,3.397895e+06,74.852273,-0.086984,-0.086425


In [15]:
sp500

,Month,Open,High,Low,Close,Adj Close,Volume,Avg_price,PC_change_price,PC_change_vol
0,2003-01-01,879.820007,935.049988,840.340027,855.700012,855.700012,30749580000,867.760010,NaN,NaN
1,2003-02-01,855.700012,864.640015,806.289978,841.150024,841.150024,25235300000,848.425018,-0.022281,-0.179329
2,2003-03-01,841.150024,895.900024,788.900024,848.179993,848.179993,30080030000,844.665008,-0.004432,0.191982
3,2003-04-01,848.179993,924.239990,847.849976,916.919983,916.919983,29669610000,882.549988,0.044852,-0.013644
4,2003-05-01,916.919983,965.380005,902.830017,963.590027,963.590027,30952100000,940.255005,0.065384,0.043226
...,...,...,...,...,...,...,...,...,...,...
242,2023-07-01,4450.479980,4607.069824,4385.049805,4588.959961,4588.959961,75063200000,4519.719971,0.047030,-0.146846
243,2023-08-01,4578.830078,4584.620117,4335.310059,4507.660156,4507.660156,86840820000,4543.245117,0.005205,0.156903
244,2023-09-01,4530.600098,4541.250000,4238.629883,4288.049805,4288.049805,73482980000,4409.324951,-0.029477,-0.153820
245,2023-10-01,4284.520020,4393.569824,4103.779785,4117.370117,4117.370117,71792290000,4200.945069,-0.047259,-0.023008


In [16]:
cpi = pd.read_csv('../CPI_Monthly.csv')
cpi.drop(np.arange(181, 185), axis=0, inplace = True)
cpi.reset_index(drop = True, inplace = True)
cpi

,DATE,CPIAUCSL_PCH
0,2005-02-01,0.41754
1,2005-03-01,0.36383
2,2005-04-01,0.31072
3,2005-05-01,-0.05163
4,2005-06-01,0.05165
...,...,...
215,2023-05-01,0.12413
216,2023-06-01,0.18035
217,2023-07-01,0.16686
218,2023-08-01,0.63119


In [17]:
econ = pd.read_csv('../Economic_Indicators_Data.csv')
gdp = np.array(econ['GDP_PCH'])

In [18]:
econ

,DATE,CPI_PCH,GDP_PCH,InterestRate_PCH,VAC_PCH,VAI_PCH
0,2005-04-01,0.67579,1.21694,19.16329,2.56574,1.42099
1,2005-07-01,1.51463,1.70233,17.55379,3.20075,-1.63202
2,2005-10-01,0.93252,1.38147,15.02890,1.00873,0.03130
3,2006-01-01,0.52075,2.06358,11.97655,4.11388,-0.87623
4,2006-04-01,0.90241,1.13436,10.09723,0.00000,2.92028
...,...,...,...,...,...,...
67,2022-01-01,2.21990,1.51904,50.00000,1.75384,-0.76782
68,2022-04-01,2.33140,2.05823,541.66667,0.16222,0.62638
69,2022-07-01,1.35832,1.76308,184.41558,2.12572,3.73490
70,2022-10-01,1.02517,1.59174,66.81887,3.57815,1.61666


In [19]:
gdp2 = np.where(gdp > 0, gdp**(1/3), 0)
gdp2

/var/folders/38/93x_6v210ts96y3j3f9njy5w0000gn/T/ipykernel_26021/221694787.py:1: RuntimeWarning: invalid value encountered in power
  gdp2 = np.where(gdp > 0, gdp**(1/3), 0)


array([1.06763562, 1.1940282 , 1.11373146, 1.27313298, 1.04291833,
       0.94688918, 1.06885661, 1.07843681, 1.09458774, 1.04006656,
       1.01198577, 0.        , 1.02670073, 0.60730876, 0.        ,
       0.        , 0.        , 0.77770749, 1.11884074, 0.91804326,
       1.13448086, 1.02519964, 1.03497888, 0.64964083, 1.10315622,
       0.83366742, 1.07534692, 1.1266258 , 0.95124122, 0.88522375,
       0.85184834, 1.11524504, 0.7848828 , 1.10408946, 1.11998724,
       0.32162584, 1.23095174, 1.17710292, 0.84612306, 0.94560108,
       1.06182846, 0.87352513, 0.56681926, 0.7897227 , 1.00091583,
       0.98886651, 1.01360085, 0.99966989, 0.93690243, 1.09318932,
       1.20492191, 1.13305462, 1.07474124, 1.0190835 , 0.8304947 ,
       0.96206586, 1.09966932, 1.13115344, 0.98623815, 0.        ,
       0.        , 2.05752224, 1.20297641, 1.37750266, 1.45222963,
       1.32073459, 1.51319034, 1.14953731, 1.2720318 , 1.20806603,
       1.16759092, 1.15338386])

In [20]:
gdp_monthly = np.repeat(gdp2, 3)

In [21]:
len(gdp_monthly)

216

In [22]:
vac = np.array(econ['VAC_PCH'])

In [23]:
vai = np.array(econ['VAI_PCH'])

In [24]:
vac2 = np.where(vac > 0, np.sign(vac)*np.abs(vac)**(1/3), 0)
vac_monthly = np.repeat(vac2, 3)

In [25]:
vai2 = np.where(vai > 0, np.sign(vai)*np.abs(vai)**(1/3), 0)
vai_monthly = np.repeat(vai2, 3)

In [26]:
ir = pd.read_csv('../InterestRate_Monthly.csv')
ir.drop(np.arange(181, 185), axis=0, inplace = True)
ir.reset_index(drop = True, inplace = True)
ir

,DATE,FEDFUNDS_PCH
0,2005-02-01,9.64912
1,2005-03-01,5.20000
2,2005-04-01,6.08365
3,2005-05-01,7.52688
4,2005-06-01,1.33333
...,...,...
216,2023-06-01,0.39526
217,2023-07-01,0.78740
218,2023-08-01,4.10156
219,2023-09-01,0.00000


In [27]:
inputs = pd.DataFrame(columns = ['Month', 'CPI', 'InterestRate', 'GDP', 'ValAddConst', 'ValAddInfo', 'Borrowing', 'CommercialLoan', 'ConsumerLoan', 'Deficit', 'ITBPrice', 'ITBVol', 'VGTPrice', 'VGTVol', 'S&P500Price', 'S&P500Vol'])

inputs['Borrowing'] = np.array(banks['borrowings_pc'][-207:])
inputs['ConsumerLoan'] = np.array(banks['consumer_loans_pc'][-207:])
inputs['CommercialLoan'] = np.array(banks['commercial_loans_pc'][-207:])
inputs['Deficit'] = np.array(deficit['Percent_change'][-207:])
inputs['ITBPrice'] = np.array(itb['PC_change_price'])
inputs['ITBVol'] = np.array(itb['PC_change_vol'])
inputs['VGTPrice'] = np.array(vgt['PC_change_price'][-207:])
inputs['VGTVol'] = np.array(vgt['PC_change_vol'][-207:])
inputs['S&P500Price'] = np.array(sp500['PC_change_price'][-207:])
inputs['S&P500Vol'] = np.array(sp500['PC_change_vol'][-207:])
inputs['Month'] = pd.to_datetime(itb['Date'])
inputs['CPI'] = np.array(cpi['CPIAUCSL_PCH'][-207:])
inputs['InterestRate'] = np.array(ir['FEDFUNDS_PCH'][-207:])
inputs['GDP'] = gdp_monthly[-207:]
inputs['ValAddConst'] = vac_monthly[-207:]
inputs['ValAddInfo'] = vai_monthly[-207:]
inputs

,Month,CPI,InterestRate,GDP,ValAddConst,ValAddInfo,Borrowing,CommercialLoan,ConsumerLoan,Deficit,ITBPrice,ITBVol,VGTPrice,VGTVol,S&P500Price,S&P500Vol
0,2006-05-01,0.15045,4.35730,1.273133,1.602325,0.000000,0.25008,11.5,4.1,-0.284777,NaN,NaN,-0.049741,0.757681,-0.012546,0.254094
1,2006-06-01,0.50075,3.13152,1.273133,1.602325,0.000000,-0.40885,17.8,5.4,-2.393523,-0.141342,0.058676,-0.058480,-0.357298,-0.015674,0.010319
2,2006-07-01,0.29895,1.01215,1.273133,1.602325,0.000000,1.64957,20.1,7.8,-1.361045,-0.087839,2.328675,-0.036031,0.028369,0.002547,-0.155359
3,2006-08-01,0.24839,5.01002,1.042918,0.000000,1.429360,0.75333,22.3,4.5,-1.478174,-0.021042,0.796079,0.032067,-0.368216,0.013990,0.089268
4,2006-09-01,0.54509,0.19084,1.042918,0.000000,1.429360,-0.89659,17.7,5.6,-2.616416,0.046412,2.381466,0.066667,0.267074,0.022189,-0.029398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2023-07-01,0.12413,0.39526,1.167591,1.529512,1.173653,0.16145,-2.6,6.0,-2.364214,0.079389,0.066784,0.040382,-0.028132,0.047030,-0.146846
203,2023-08-01,0.18035,0.78740,1.167591,1.529512,1.173653,-1.63324,-2.1,3.0,-0.052340,-0.008281,0.420013,-0.034826,-0.024273,0.005205,0.156903
204,2023-09-01,0.16686,4.10156,1.153384,1.176099,1.263007,-3.31905,-0.7,2.4,-0.030673,-0.047472,0.073404,-0.009513,-0.191162,-0.029477,-0.153820
205,2023-10-01,0.63119,0.00000,1.153384,1.176099,1.263007,-1.04501,-0.1,5.9,-1.404270,-0.086984,-0.086425,-0.025103,0.135212,-0.047259,-0.023008


In [28]:
inputs.iloc[:, 1:9] = inputs.iloc[:, 1:9]/100
inputs

,Month,CPI,InterestRate,GDP,ValAddConst,ValAddInfo,Borrowing,CommercialLoan,ConsumerLoan,Deficit,ITBPrice,ITBVol,VGTPrice,VGTVol,S&P500Price,S&P500Vol
0,2006-05-01,0.001504,0.043573,0.012731,0.016023,0.000000,0.002501,0.115,0.041,-0.284777,NaN,NaN,-0.049741,0.757681,-0.012546,0.254094
1,2006-06-01,0.005008,0.031315,0.012731,0.016023,0.000000,-0.004089,0.178,0.054,-2.393523,-0.141342,0.058676,-0.058480,-0.357298,-0.015674,0.010319
2,2006-07-01,0.002989,0.010122,0.012731,0.016023,0.000000,0.016496,0.201,0.078,-1.361045,-0.087839,2.328675,-0.036031,0.028369,0.002547,-0.155359
3,2006-08-01,0.002484,0.050100,0.010429,0.000000,0.014294,0.007533,0.223,0.045,-1.478174,-0.021042,0.796079,0.032067,-0.368216,0.013990,0.089268
4,2006-09-01,0.005451,0.001908,0.010429,0.000000,0.014294,-0.008966,0.177,0.056,-2.616416,0.046412,2.381466,0.066667,0.267074,0.022189,-0.029398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2023-07-01,0.001241,0.003953,0.011676,0.015295,0.011737,0.001615,-0.026,0.060,-2.364214,0.079389,0.066784,0.040382,-0.028132,0.047030,-0.146846
203,2023-08-01,0.001804,0.007874,0.011676,0.015295,0.011737,-0.016332,-0.021,0.030,-0.052340,-0.008281,0.420013,-0.034826,-0.024273,0.005205,0.156903
204,2023-09-01,0.001669,0.041016,0.011534,0.011761,0.012630,-0.033190,-0.007,0.024,-0.030673,-0.047472,0.073404,-0.009513,-0.191162,-0.029477,-0.153820
205,2023-10-01,0.006312,0.000000,0.011534,0.011761,0.012630,-0.010450,-0.001,0.059,-1.404270,-0.086984,-0.086425,-0.025103,0.135212,-0.047259,-0.023008


In [31]:
inputs

,Month,CPI,InterestRate,GDP,ValAddConst,ValAddInfo,Borrowing,CommercialLoan,ConsumerLoan,Deficit,ITBPrice,ITBVol,VGTPrice,VGTVol,S&P500Price,S&P500Vol
1,2006-06-01,0.005008,0.031315,0.012731,0.016023,0.000000,-0.004089,0.178,0.054,-2.393523,-0.141342,0.058676,-0.058480,-0.357298,-0.015674,0.010319
2,2006-07-01,0.002989,0.010122,0.012731,0.016023,0.000000,0.016496,0.201,0.078,-1.361045,-0.087839,2.328675,-0.036031,0.028369,0.002547,-0.155359
3,2006-08-01,0.002484,0.050100,0.010429,0.000000,0.014294,0.007533,0.223,0.045,-1.478174,-0.021042,0.796079,0.032067,-0.368216,0.013990,0.089268
4,2006-09-01,0.005451,0.001908,0.010429,0.000000,0.014294,-0.008966,0.177,0.056,-2.616416,0.046412,2.381466,0.066667,0.267074,0.022189,-0.029398
5,2006-10-01,0.004436,0.000000,0.010429,0.000000,0.014294,-0.005700,0.222,0.074,0.951423,0.046867,-0.436061,0.044646,0.280520,0.028076,0.159019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,2023-05-01,0.000530,0.038710,0.012081,0.012858,0.015515,0.166242,0.013,0.073,0.440654,0.052962,0.067876,0.038191,-0.165449,0.009060,0.254976
201,2023-06-01,0.003678,0.047619,0.011676,0.015295,0.011737,0.062205,-0.029,0.058,-1.465993,0.066633,-0.030136,0.092126,0.350401,0.034360,-0.010638
202,2023-07-01,0.001241,0.003953,0.011676,0.015295,0.011737,0.001615,-0.026,0.060,-2.364214,0.079389,0.066784,0.040382,-0.028132,0.047030,-0.146846
203,2023-08-01,0.001804,0.007874,0.011676,0.015295,0.011737,-0.016332,-0.021,0.030,-0.052340,-0.008281,0.420013,-0.034826,-0.024273,0.005205,0.156903


In [30]:
inputs = inputs[1:-2]
inputs

,Month,CPI,InterestRate,GDP,ValAddConst,ValAddInfo,Borrowing,CommercialLoan,ConsumerLoan,Deficit,ITBPrice,ITBVol,VGTPrice,VGTVol,S&P500Price,S&P500Vol
1,2006-06-01,0.005008,0.031315,0.012731,0.016023,0.000000,-0.004089,0.178,0.054,-2.393523,-0.141342,0.058676,-0.058480,-0.357298,-0.015674,0.010319
2,2006-07-01,0.002989,0.010122,0.012731,0.016023,0.000000,0.016496,0.201,0.078,-1.361045,-0.087839,2.328675,-0.036031,0.028369,0.002547,-0.155359
3,2006-08-01,0.002484,0.050100,0.010429,0.000000,0.014294,0.007533,0.223,0.045,-1.478174,-0.021042,0.796079,0.032067,-0.368216,0.013990,0.089268
4,2006-09-01,0.005451,0.001908,0.010429,0.000000,0.014294,-0.008966,0.177,0.056,-2.616416,0.046412,2.381466,0.066667,0.267074,0.022189,-0.029398
5,2006-10-01,0.004436,0.000000,0.010429,0.000000,0.014294,-0.005700,0.222,0.074,0.951423,0.046867,-0.436061,0.044646,0.280520,0.028076,0.159019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,2023-05-01,0.000530,0.038710,0.012081,0.012858,0.015515,0.166242,0.013,0.073,0.440654,0.052962,0.067876,0.038191,-0.165449,0.009060,0.254976
201,2023-06-01,0.003678,0.047619,0.011676,0.015295,0.011737,0.062205,-0.029,0.058,-1.465993,0.066633,-0.030136,0.092126,0.350401,0.034360,-0.010638
202,2023-07-01,0.001241,0.003953,0.011676,0.015295,0.011737,0.001615,-0.026,0.060,-2.364214,0.079389,0.066784,0.040382,-0.028132,0.047030,-0.146846
203,2023-08-01,0.001804,0.007874,0.011676,0.015295,0.011737,-0.016332,-0.021,0.030,-0.052340,-0.008281,0.420013,-0.034826,-0.024273,0.005205,0.156903


In [32]:
inputs.to_csv('../all_inputs_cleaned.csv')

In [23]:
outputs = pd.DataFrame(columns = ['Month', 'Construction', 'Information', 'Total_Private'])

outputs['Construction'] = np.array(employment['Construction'][-205:])
outputs['Information'] = np.array(employment['Information'][-205:])
outputs['Total_Private'] = np.array(employment['Total private'][-205:])
outputs['Month'] = pd.to_datetime(np.array(employment['Month'][-205:]))
outputs

,Month,Construction,Information,Total_Private
0,2006-05-01,-0.001683,-0.002626,0.000254
1,2006-06-01,-0.001815,-0.000987,0.000744
2,2006-07-01,0.001689,-0.001647,0.001250
3,2006-08-01,0.001037,0.001320,0.001117
4,2006-09-01,-0.000259,-0.002306,0.000567
...,...,...,...,...
200,2023-05-01,0.003164,-0.001292,0.001916
201,2023-06-01,0.003659,-0.003235,0.000645
202,2023-07-01,0.001508,-0.006167,0.001087
203,2023-08-01,0.004519,-0.006858,0.001325


In [24]:
outputs.to_csv('../all_outputs_cleaned.csv')